# Swim TCSP Import CSVs

In [3]:
# pip install pymysql

## Connection and Test

In [4]:
import csv
import pymysql

def connect_to_tcsp():
    connection_details = {
        "host": 'tcsp.ie',
        "port": 3306,
        "user": 't567715',
        "password": '0bjs8Pz55Q',
        "database": 't567715_wp_tcsp',
        "charset": 'utf8mb4',
        "cursorclass": pymysql.cursors.DictCursor
    }
    try:
        connection = pymysql.connect(**connection_details)
        return connection
    except pymysql.MySQLError as e:
        print(f"Error connecting to MySQL Database: {e}")
        return None



In [5]:
def fetch_mor_events(connection):
    if connection is None:
        return []
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT id, event FROM mor_events")
            return cursor.fetchall()
    except pymysql.MySQLError as e:
        print(f"Error executing query: {e}")
        return []

def save_data_to_csv(data, filename):
    if data:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=data[0].keys())
            writer.writeheader()
            writer.writerows(data)

# Main execution
try:
    conx = connect_to_tcsp()
    if conx is not None:
        data = fetch_mor_events(conx)
        save_data_to_csv(data, 'public_swim_catagories.csv')
finally:
    if conx:
        conx.close()


## Get field names
This is used to get a list of field names for the processes below

In [6]:
import pymysql

def get_table_field_names(connection, table_name):
    if connection is None:
        return []
    try:
        with connection.cursor(pymysql.cursors.DictCursor) as cursor:  # Use DictCursor
            cursor.execute(f"DESCRIBE {table_name}")
            # Extracting only the column names using DictCursor
            return [row["Field"] for row in cursor.fetchall()]  # Adjusted to access 'Field' key
    except pymysql.MySQLError as e:
        print(f"Error executing query: {e}")
        return []

# Example usage
try:
    conx = connect_to_tcsp()  # Replace with your actual connection setup function
    if conx is not None:
        field_names = get_table_field_names(conx, 'mor_generic_bookings')
        print(field_names)
finally:
    if 'conx' in locals() and conx:
        conx.close()



['id', 'customer_id', 'session_id', 'session_date', 'wc_order_id', 'order_type', 'booking_date', 'num_party', 'num_adults', 'num_children', 'num_senior', 'num_under3', 'num_guests', 'covid_check1', 'covid_check2', 'covid_check3', 'paid', 'attended', 'notes', 'adult_price', 'child_price', 'senior_price', 'under3_price', 'event']


## More Generic (Supply sql and File names pairs)

In [21]:
import csv
import pymysql

def execute_query(connection, query):
    if connection is None:
        return []
    try:
        with connection.cursor(pymysql.cursors.DictCursor) as cursor:  # Use DictCursor
            cursor.execute(query)
            return cursor.fetchall()
    except pymysql.MySQLError as e:
        print(f"Error executing query: {e}")
        return []

def save_data_to_csv(data, filename):
    if data:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=data[0].keys())
            writer.writeheader()
            writer.writerows(data)

# Queries and corresponding CSV file names
#Public Swims
column_names_public_swims = ['id', 'day_id', 'event_id', 'num_places', 'time_start', 'time_end', 'notes', 'active']
query_public_swims = f"SELECT {', '.join(column_names_public_swims)} FROM mor_sessions_generic"
# Orders
column_names_orders = ['wc_order_id as id', 'customer_id as user_id', 'session_id as product_id', 'session_date as booking ', 'wc_order_id as stripe_id']
query_orders = f"SELECT {', '.join(column_names_orders)} FROM mor_generic_bookings WHERE booking_date > '2023-11-25 " \
               f"12:16:17' AND wc_order_id IS NOT NULL ;"
# Order Items
column_names_order_items = ['wc_order_id as id', 'num_adults', 'num_children', 'num_senior', 'num_under3']

query_order_items= f"SELECT {', '.join(column_names_order_items)} FROM mor_generic_bookings WHERE booking_date > " \
                  f"'2023-11-25 " \
               f"12:16:17' AND wc_order_id IS NOT NULL ;"

#Data for query
query_file_pairs = [
    # public_swim_categories
    ("SELECT id, event FROM mor_events", "public_swim_categories.csv"),
    # terms
    ("SELECT start_date, finish_date, COALESCE(rebook_start, '2000-01-01') AS rebook_start, COALESCE(booking_switch_date, '2000-01-01') AS booking_switch_date, COALESCE(assesments_complete, '2000-01-01') AS assesments_complete FROM mor_terms", "terms.csv"),
    # Public Swims
    (query_public_swims, "public_swims.csv"),
    # Public Swims Orders
    (query_orders, "public_swims_orders.csv"),
    # Public Swims Order items
    (query_order_items, "public_swims_order_items.csv"),
]

# Main execution
try:
    conx = connect_to_tcsp()  # Replace with your actual connection setup function
    if conx is not None:
        for query, filename in query_file_pairs:
            data = execute_query(conx, query)
            save_data_to_csv(data, filename)
finally:
    if 'conx' in locals() and conx:
        conx.close()


# Extract User Data
The data is all extracted

In [8]:
import csv
import pymysql

def connect_to_tcsp():
    connection_details = {
        "host": 'tcsp.ie',
        "port": 3306,
        "user": 't567715',
        "password": '0bjs8Pz55Q',
        "database": 't567715_wp_tcsp',
        "charset": 'utf8mb4',
        "cursorclass": pymysql.cursors.DictCursor
    }
    try:
        connection = pymysql.connect(**connection_details)
        return connection
    except pymysql.MySQLError as e:
        print(f"Error connecting to MySQL Database: {e}")
        return None

# SQL query with WHERE clause to filter users with ID greater than 13149
query = '''
SELECT
    u1.ID AS user_id,
    u1.user_email AS user_email,
    u1.user_login AS username,
    m4.meta_value AS mobile_phone,
    m7.meta_value AS user_phone,
    m5.meta_value AS Role,
    m8.meta_value AS notes,
    m9.meta_value AS other_phone,
    m10.meta_value AS first_name,
    m11.meta_value AS last_name
FROM wpmor_users u1
LEFT JOIN wpmor_usermeta m4 ON m4.user_id = u1.ID AND m4.meta_key = 'mobile'
LEFT JOIN wpmor_usermeta m5 ON m5.user_id = u1.ID AND m5.meta_key = 'wpmor_capabilities'
LEFT JOIN wpmor_usermeta m7 ON m7.user_id = u1.ID AND m7.meta_key = 'user_phone'
LEFT JOIN wpmor_usermeta m8 ON m8.user_id = u1.ID AND m8.meta_key = 'description'
LEFT JOIN wpmor_usermeta m9 ON m9.user_id = u1.ID AND m9.meta_key = 'billing_phone'
LEFT JOIN wpmor_usermeta m10 ON m10.user_id = u1.ID AND m10.meta_key = 'first_name'
LEFT JOIN wpmor_usermeta m11 ON m11.user_id = u1.ID AND m11.meta_key = 'last_name'
WHERE u1.ID > 13149;
'''

connection = connect_to_tcsp()
if connection:
    try:
        with connection.cursor() as cursor:
            cursor.execute(query)

            with open('users.csv', 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                # Write the header
                writer.writerow([i[0] for i in cursor.description])
                # Write the data
                for row in cursor:
                    writer.writerow(row.values())
    finally:
        connection.close()


## New Users Only

In [20]:
# This is to get new users from a certain user id

import csv
import pymysql

def connect_to_tcsp():
    connection_details = {
        "host": 'tcsp.ie',
        "port": 3306,
        "user": 't567715',
        "password": '0bjs8Pz55Q',
        "database": 't567715_wp_tcsp',
        "charset": 'utf8mb4',
        "cursorclass": pymysql.cursors.DictCursor
    }
    try:
        connection = pymysql.connect(**connection_details)
        return connection
    except pymysql.MySQLError as e:
        print(f"Error connecting to MySQL Database: {e}")
        return None

# SQL query with WHERE clause to filter users with ID greater than 13149
query = '''
SELECT
    u1.ID AS user_id,
    u1.user_email AS user_email,
    u1.user_login AS username,
    m4.meta_value AS mobile_phone,
    m7.meta_value AS user_phone,
    m5.meta_value AS Role,
    m8.meta_value AS notes,
    m9.meta_value AS other_phone,
    m10.meta_value AS first_name,
    m11.meta_value AS last_name
FROM wpmor_users u1
LEFT JOIN wpmor_usermeta m4 ON m4.user_id = u1.ID AND m4.meta_key = 'mobile'
LEFT JOIN wpmor_usermeta m5 ON m5.user_id = u1.ID AND m5.meta_key = 'wpmor_capabilities'
LEFT JOIN wpmor_usermeta m7 ON m7.user_id = u1.ID AND m7.meta_key = 'user_phone'
LEFT JOIN wpmor_usermeta m8 ON m8.user_id = u1.ID AND m8.meta_key = 'description'
LEFT JOIN wpmor_usermeta m9 ON m9.user_id = u1.ID AND m9.meta_key = 'billing_phone'
LEFT JOIN wpmor_usermeta m10 ON m10.user_id = u1.ID AND m10.meta_key = 'first_name'
LEFT JOIN wpmor_usermeta m11 ON m11.user_id = u1.ID AND m11.meta_key = 'last_name'
WHERE u1.ID >= 13146;
'''

connection = connect_to_tcsp()
if connection:
    try:
        with connection.cursor() as cursor:
            cursor.execute(query)

            with open('new_users.csv', 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                # Write the header
                writer.writerow([i[0] for i in cursor.description])
                # Write the data
                for row in cursor:
                    writer.writerow(row.values())
    finally:
        connection.close()


In [10]:
# pip install phpserialize

## This program takes out the roles from the user data which is exported to users.csv from the database(TCSP) and creats a new file roles_output with the roles deserilised ro comma separated groups.

In [11]:
# Outputs new_users to new roles
import csv
import phpserialize

input_file = 'new_users.csv'  # Update the path if needed
output_file = 'new_roles_output.csv'

def deserialize_php(serialized_php):
    try:
        return phpserialize.loads(serialized_php.encode(), decode_strings=True)
    except Exception as e:
        print(f"Error deserializing: {e}")
        return None

def process_data(input_file, output_file):
    with open(input_file, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        fieldnames = [name for name in reader.fieldnames if name != 'Role'] + ['groups']

        with open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            processed_emails = set()  # Set to store processed emails

            for row in reader:
                email = row.get('email', '').strip().lower()
                if email in processed_emails:
                    continue  # Skip this row as email is already processed

                serialized_role_data = row.get('Role')
                groups = 'customer'  # Default value for groups
                if serialized_role_data:
                    deserialized_data = deserialize_php(serialized_role_data)
                    if deserialized_data and any(deserialized_data.values()):
                        groups = ', '.join([role for role, value in deserialized_data.items() if value])

                new_row = {key: value for key, value in row.items() if key != 'Role'}
                new_row['groups'] = groups

                writer.writerow(new_row)
                processed_emails.add(email)  # Add email to the processed set

process_data(input_file, output_file)



In [19]:
# Takes new user list and changes roles to groups
import csv
import phpserialize
import re

input_file = 'new_users.csv'  # Update the path if needed
output_file = 'new_roles_output.csv'

def is_valid_email(email):
    if email.startswith('**') or email.startswith('.'):
        return False
    return True

def deserialize_php(serialized_php):
    try:
        return phpserialize.loads(serialized_php.encode(), decode_strings=True)
    except Exception as e:
        print(f"Error deserializing: {e}")
        return None

def process_data(input_file, output_file):
    with open(input_file, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        # Adjust fieldnames, drop 'Role' and 'user_phone', add 'groups'
        fieldnames = ['id' if name == 'user_id' else 'email' if name == 'user_email' else name
                      for name in reader.fieldnames if name not in ['Role', 'user_phone']] + ['groups']

        with open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                user_email = row.get('user_email')
                if not is_valid_email(user_email):
                    continue  # Skip rows where the email is not valid

                serialized_role_data = row.get('Role')
                groups = 'customer'  # Default value for groups
                if serialized_role_data:
                    deserialized_data = deserialize_php(serialized_role_data)
                    if deserialized_data and any(deserialized_data.values()):
                        groups = ', '.join([role for role, value in deserialized_data.items() if value])

                # Create a new row dictionary without 'Role' and 'user_phone', rename 'user_id' and 'user_email'
                new_row = {('id' if key == 'user_id' else 'email' if key == 'user_email' else key): value
                           for key, value in row.items() if key not in ['Role', 'user_phone']}
                new_row['groups'] = groups

                writer.writerow(new_row)

process_data(input_file, output_file)



## Batch Version
Used when all users are beig imported

In [13]:
import csv
import phpserialize
import itertools

input_file = 'users.csv'  # Update the path if needed

def is_valid_email(email):
    # Basic validation to check if the email doesn't start with '**' or '.'
    if email.startswith('**') or email.startswith('.'):
        return False
    return True

def deserialize_php(serialized_php):
    try:
        return phpserialize.loads(serialized_php.encode(), decode_strings=True)
    except Exception as e:
        print(f"Error deserializing: {e}")
        return None

def process_data(input_file, output_files):
    with open(input_file, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        fieldnames = [name for name in reader.fieldnames if name != 'Role'] + ['groups']

        writers = []
        outfiles = []
        for file in output_files:
            outfile = open(file, mode='w', newline='', encoding='utf-8')
            outfiles.append(outfile)
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()
            writers.append(writer)

        writer_cycle = itertools.cycle(writers)

        for row in reader:
            user_email = row.get('user_email')
            if not is_valid_email(user_email):
                continue

            serialized_role_data = row.get('Role')
            groups = 'customer'
            if serialized_role_data:
                deserialized_data = deserialize_php(serialized_role_data)
                if deserialized_data and any(deserialized_data.values()):
                    groups = ', '.join([role for role, value in deserialized_data.items() if value])

            new_row = {key: value for key, value in row.items() if key != 'Role'}
            new_row['groups'] = groups

            next(writer_cycle).writerow(new_row)

    # Close all output files
    for outfile in outfiles:
        outfile.close()

# List of output files
output_files = ['roles_output_batch1.csv', 'roles_output_batch2.csv', 'roles_output_batch3.csv', 'roles_output_batch4.csv']

process_data(input_file, output_files)


## Updated Version of batch - For all users

In [14]:
import csv
import phpserialize
import itertools

input_file = 'users.csv'  # Update the path if needed

def is_valid_email(email):
    if email.startswith('**') or email.startswith('.'):
        return False
    return True

def deserialize_php(serialized_php):
    try:
        return phpserialize.loads(serialized_php.encode(), decode_strings=True)
    except Exception as e:
        print(f"Error deserializing: {e}")
        return None

def process_data(input_file, output_files):
    seen_emails = set()  # Set to track already processed emails

    with open(input_file, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        fieldnames = ['id' if name == 'user_id' else ('email' if name == 'user_email' else name) for name in reader.fieldnames if name != 'Role'] + ['groups']

        writers = []
        outfiles = []
        for file in output_files:
            outfile = open(file, mode='w', newline='', encoding='utf-8')
            outfiles.append(outfile)
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()
            writers.append(writer)

        writer_cycle = itertools.cycle(writers)

        for row in reader:
            user_email = row.get('user_email')
            if not is_valid_email(user_email) or user_email in seen_emails:
                continue  # Skip invalid emails and duplicates

            seen_emails.add(user_email)  # Add email to the set of seen emails

            serialized_role_data = row.get('Role')
            groups = 'customer'
            if serialized_role_data:
                deserialized_data = deserialize_php(serialized_role_data)
                if deserialized_data and any(deserialized_data.values()):
                    groups = ', '.join([role for role, value in deserialized_data.items() if value])

            new_row = {('id' if key == 'user_id' else ('email' if key == 'user_email' else key)): value for key, value in row.items() if key != 'Role'}
            new_row['groups'] = groups

            next(writer_cycle).writerow(new_row)

    for outfile in outfiles:
        outfile.close()

output_files = ['roles_output_batch1.csv', 'roles_output_batch2.csv', 'roles_output_batch3.csv', 'roles_output_batch4.csv']
process_data(input_file, output_files)


# Order Items
Used to transpose Order Items to import into Django

## Split Orderitems

In [15]:
import csv

def transform_csv(input_csv_path, output_csv_path):
    with open(input_csv_path, mode='r', newline='', encoding='utf-8') as infile, \
         open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=['id', 'type', 'quantity'])
        writer.writeheader()

        for row in reader:
            id = row['id']
            for key in ['num_adults', 'num_children', 'num_senior', 'num_under3']:
                type_name = key.split('_')[1]
                for _ in range(int(row[key])):
                    writer.writerow({'id': id, 'type': type_name, 'quantity': 1})

# Replace 'input_csv_path' and 'output_csv_path' with your actual file paths
input_csv_path = 'public_swims_order_items.csv'
output_csv_path = 'public_swims_order_items_transposed.csv'

transform_csv(input_csv_path, output_csv_path)


## Changes Order Items to the transposed version

In [22]:
# This works

import csv

def transform_csv(input_csv_path, output_csv_path):
    with open(input_csv_path, mode='r', newline='', encoding='utf-8') as infile, \
         open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=['id', 'variant', 'quantity'])
        writer.writeheader()

        # Mapping from original field names to new variant names
        variant_mappings = {
            'num_adults': 'Adult',
            'num_children': 'Child',
            'num_senior': 'OAP',
            'num_under3': 'Infant'
        }

        for row in reader:
            id = row['id']
            for key, new_variant_name in variant_mappings.items():
                quantity = int(row[key])
                if quantity > 0:
                    writer.writerow({'id': id, 'variant': new_variant_name, 'quantity': quantity})

# Replace 'input_csv_path' and 'output_csv_path' with your actual file paths
input_csv_path = 'public_swims_order_items.csv'
output_csv_path = 'public_swims_order_items_transposed.csv'

transform_csv(input_csv_path, output_csv_path)

